In [ ]:
#this is not a logically working code

In [6]:
import gc
import os
import torch
from pathlib import Path
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Configuration
IMG_DIR = "/Volumes/JavaAOT/Documents/AI/ml_expressions/img_datasets/ferckja_dataset/disgust copy 2"
BATCH_SIZE = 8  # Adjust based on your system's capacity
SYNTHETIC_THRESHOLD = 0.5

# Load model and processor
model_name = "google/efficientnet-b0"  # Replace with the chosen model
model = AutoModelForImageClassification.from_pretrained(model_name).eval()
processor = AutoImageProcessor.from_pretrained(model_name)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to process a batch of images
def process_batch(image_paths):
    images = [Image.open(path).convert("RGB") for path in image_paths]
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs

image_paths = list(Path(IMG_DIR).glob("*.jpg"))  # Adjust the pattern as needed
synthetic_images = []

for i in range(0, len(image_paths), BATCH_SIZE):
    batch_paths = image_paths[i:i + BATCH_SIZE]
    probs = process_batch(batch_paths)
    for path, prob in zip(batch_paths, probs):
        fake_prob = prob[1].item()  # Assuming index 1 corresponds to 'fake'
        if fake_prob >= SYNTHETIC_THRESHOLD:
            synthetic_images.append(path)
    # Free up memory
    del probs
    torch.cuda.empty_cache()
    gc.collect()

# Delete synthetic images
for path in synthetic_images:
    try:
        os.remove(path)
        print(f"Deleted synthetic image: {path}")
    except Exception as e:
        print(f"Failed to delete {path}: {e}")

print(f"Total synthetic images removed: {len(synthetic_images)}")

config.json:   0%|          | 0.00/70.3k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

Total synthetic images removed: 0
